## Imports for the Project

In [11]:
import torch
import torchvision
from torchvision import transforms , datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchsummary import summary # for model summary
import matplotlib.pyplot as plt # for ploting our data and showing images
import numpy as np
import cv2
import dataHelper as dH
import os
import time
import ipywidgets
from IPython.display import display
from jupyter_clickable_image_widget import ClickableImageWidget

## Model Setup

Here we are letting pytorch know to use the gpu and we are also importing a pretrained resnet18 model that we are modifying to return only 2 outputs. One for X and one for Y.

In [ ]:
device = torch.device('cuda')
output_dim = 2
# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)
model = model.to(device)

## Model Summary
This is the inner workings of resnet18.

In [ ]:
summary(model, input_size=(3,224,224))

## Data Collection

### Files Structure Setup

In [ ]:
path_raw = os.getcwd() + '/raw_datasets'
try:
    os.mkdir(path_raw)
except OSError:
    pass
else:
    pass

In [ ]:
#config
datacollect = False
dataSetSize = 5
FPScollect = 30
SEC_delay = 2


loopcounter = 0
collectedNum = 0

if datacollect == True:
    
    cap = cv2.VideoCapture(0)
    print('Get Ready!')
    cv2.imshow('frame',frame)
    cv2.imshow('crop', dH.imgcrop224(frame))
    time.sleep(6)
    print('Begin!')
    
    while True:
        if collectedNum == dataSetSize:
            break
        ret, frame = cap.read()
        cv2.imshow('frame',frame)
        if loopcounter % (FPScollect * SEC_delay) == 0:
            cv2.imshow('crop', dH.imgcrop224(frame))
            # write raw images to file 
            dH.write_raw_frame(path_raw, dH.imgcrop224(frame), collectedNum)
            collectedNum += 1
        loopcounter += 1
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

            
print('Done!')
cap.release()
cv2.destroyAllWindows()

## Data Labeling

In [20]:
path_labeled = os.getcwd() + '/labeled_datasets'
try:
    os.mkdir(path_labeled)
except OSError:
    pass
else:
    pass

def on_click(_, content,msg):
        if content['event'] == 'click':
            data = content['eventData']
            x = data['offsetX']
            y = data['offsetY']
            print(f'{x},{y}')

In [25]:
datalabel = True

if datalabel == True:
    with open(path_raw + '/' +'img_raw.0.jpg', "rb") as f:
        image = f.read()
        click_widget = ClickableImageWidget(
            value=image,
            width=224,
            height=224)
    
    with open(path_raw + '/' +'img_raw.1.jpg', "rb") as f:
        image = f.read()
        w = ipywidgets.Image(
            value=image,
            width=224,
            height=224)

    click_widget.on_msg(on_click)
    #display(ipywidgets.HBox([w]))
    display(w)
    display(click_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

ClickableImageWidget(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C…